# Lab 4, Zad 2
## Obraz binarny

Celem ćwiczenia jest zobrazowanie działania algorytmu wyżarzania w kontekście różnych funkcji kosztu, aby zrozumieć, jak zmienia się jego skuteczność w zależności od charakterystyki problemu.

# Importy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from PIL import Image

# Funkcje przydatne do wizualizacji

Funckcja `create_image_from_array` tworzy obraz z macierzy

In [ ]:
def create_image_from_array(matrix):
    # Pillow interpretuje 0 jako czarny, 255 jako biały w trybie 'L' (8-bit grayscale)
    arr = (1 - matrix.astype(np.uint8)) * 255  # 1 -> 0 (czarny), 0 -> 255 (biały)
    return Image.fromarray(arr, mode='L')


Funkcja `draw_plot(array)`  rysuje obraz z macierzy

In [ ]:
def draw_plot(matrix, path = None, folder_name = ''):
    n = len(matrix)
    img = Image.new('1', (n, n))  # Tworzymy obraz o wymiarach NxN, gdzie '1' oznacza obraz w odcieniach szarości

    # Zmiana pikseli na podstawie tablicy
    pixels = img.load()
    for i in range(n):
        for j in range(n):
            pixels[j, i] = int(matrix[i, j])  # 0 to biały, 1 to czarny

    # Zapisywanie obrazu
    img.save(f"zad2_images/images/{folder_name}/{path}.png")

Funkcja `extend_gif(matrix,images)` dodaje kolejną klatke do gifu

In [269]:
def extend_gif(matrix,images):
    n = len(matrix)
    img = Image.new('1', (n, n))  # '1' oznacza obraz czarno-biały
    pixels = img.load()
    for i in range(n):
        for j in range(n):
            pixels[j, i] = int(matrix[i, j])
            
    images.append(img) # 0 to biały, 1 to czarny
    return images

Funkcja `draw_linear_plot` rysuje wykres liniowy z tablicy energii

In [ ]:
def draw_linear_plot(energy_tab, file_name = None, folder_name = ''):
    plt.figure()
    plt.title(f"Funkcja kosztu dla {file_name}")
    plt.xlabel('Liczba iterazcji')
    plt.ylabel('Wyliczona funkcja')
    plt.plot(energy_tab)
    if file_name:
        plt.savefig(f"zad2_images/plots/{folder_name}/{file_name}.png")
    plt.close()

# Rozwiązanie

 Do modelu wykorzystuje funkcje temperatury opratą o ciąg geometryczny

In [ ]:
def update_temp_geo(Temp, a, k ,temp_start):
    return a * Temp

Funkcja prawdopodobieństwa z zadania 1

In [18]:
def propability(diffrance, Temp):
    if diffrance < 0:
        return 1
    return np.exp(-diffrance/ Temp)

Funkcja `get_random_points(all_cords, n)` zwaraca dwie tablice losowych kordynatów punktów w macierzy o wielkości `n`

Funckja `get_all_cords(matrix)` zwraca tablice krotek indeksów tablicy

In [ ]:
def get_random_points(all_cords, n):
        return random.sample(all_cords, n), random.sample(all_cords, n)

def get_all_cords(matrix):
        return [(x, y) for x in range(len(matrix)) for y in range(len(matrix))]

Funkcja `calc_total_energy` oblicza całkowitą energię układu na podstawie podanej funkcji `calc_for_point`, która definiuje sposób oceny pojedynczej komórki. Dla każdej komórki w macierzy wynik tej funkcji jest sumowany, tworząc globalną wartość energii.

Funkcja `calc_diff_energy` zwraca różnicę w energii układu wynikającą z zamiany miejscami dwóch punktów: (x1, y1) i (x2, y2). Wykorzystuje przy tym tę samą funkcję `calc_for_point`, by oszacować zmianę lokalnej energii przed i po zamianie.

Obie funkcje pełnią rolę uniwersalnego interfejsu, który ułatwia testowanie różnych funkcji optymalizujących bez konieczności modyfikowania głównej logiki algorytmu.

In [212]:
def calc_total_energy(M, calc_for_point):
    n = len(M)
    total = 0
    for i in range(n):
        for j in range(n):
            total += calc_for_point(M, i, j, M[i][j])
    return total // 2  # unikamy podwójnego liczenia

def calc_diff_energy(M, x1, y1, x2, y2, calc_for_point):
    sum = 0
    sum -= calc_for_point(M, x1, y1, M[x1][y1])
    sum -= calc_for_point(M, x2, y2, M[x2][y2])
    sum += calc_for_point(M, x1, y1, M[x2][y2])
    sum += calc_for_point(M, x2, y2, M[x1][y1])
    return sum

## Główna funkcja 

Funkcja `create_cristal` realizuje proces optymalizacji przy użyciu algorytmu symulowanego wyżarzania (simulated annealing) na macierzy binarnej (z wartościami 0 i 1), której celem jest minimalizacja zadanej funkcji celu dla pojedynczych komórek (fun_to_optymalize). Dodatkowo może tworzyć animowany GIF ilustrujący przebieg optymalizacji.

Argumenty:
- matrix (np.ndarray): Macierz binarna (0 i 1), która reprezentuje stan układu.

- fun_to_optymalize (callable): Funkcja celu dla pojedynczej komórki, wykorzystywana do obliczenia energii układu.

- temp_start (float): Początkowa temperatura procesu wyżarzania.

- loop_limit (int): Liczba iteracji algorytmu.

- min_heat (float) (domyślnie: 1e-2): Minimalna temperatura, poniżej której proces zostaje zatrzymany.

- fun_change_par (float) (domyślnie: 0.99993): Współczynnik zmniejszania temperatury.

- fun_temp_change (callable) (domyślnie: update_temp_geo): Funkcja aktualizująca temperaturę (np. wykładnicze chłodzenie).

- gif_name (str | None): Nazwa pliku GIF-a, jeśli animacja ma zostać zapisana.

Działanie:
- Inicjalizuje temperaturę, energię układu i historię wartości funkcji celu.

- W każdej iteracji losuje parę punktów 0 i 1, oblicza różnicę energii i z prawdopodobieństwem zależnym od temperatury decyduje o akceptacji zmiany.

- Jeśli zostanie osiągnięte nowe najlepsze rozwiązanie, zapisuje je.

- Co pewną liczbę iteracji (100 równomiernie rozłożonych klatek) dodaje stan macierzy do listy klatek GIF-a.

- Aktualizuje temperaturę zgodnie z podaną funkcją.

- Na końcu zapisuje animację GIF i zwraca końcową macierz, historię energii oraz najlepszą wartość funkcji celu.

Zwraca:
- matrix (np.ndarray): Zoptymalizowana macierz.

- optymalized_tab (list[float]): Historia wartości funkcji celu.

- best_val (float): Najlepsza uzyskana wartość funkcji celu.

Następny stan jest uzyskiwany poprzez zamianę co najmniej ${2n}/100$ punktów między sobą. Proces ten zależy również od temperatury – im wyższa temperatura, tym więcej punktów zmienia się jednocześnie. Taki sposób modyfikacji pozytywnie wpływa na dynamikę tworzenia rysunku, wprowadzając pewną zmienność przy minimalnym wpływie na jego dokładność. Tego typu fluktuacje dobrze odwzorowują rzeczywiste procesy krystalizacji, gdzie asymetrie, niedoskonałości i inne nieregularności są nieodłączną częścią struktury.

Dodatkowo dla przyspieszenia generowania obrazów nie zapisuje najlepszego stanu. Zwracam ostatni stan jaki był w pętli. Przyspieszyło to algorytm kilkukrotnie

In [ ]:
def create_cristal(matrix,
                 fun_to_optymalize,
                 temp_start,
                 loop_limit,
                 min_heat=1e-2,
                 fun_change_par=0.99993,
                 fun_temp_change=update_temp_geo,
                 gif_name=None,
                ):
    
    # biore zawsze 100 klatek
    checkpoints_for_gif = np.linspace(0, loop_limit, 100)
    checkpoints_for_gif = np.round(checkpoints_for_gif).astype(int)
    eps = 10**(-8)
    temp = temp_start
    all_cords = get_all_cords(matrix)
    best_val = calc_total_energy(matrix,fun_to_optymalize)  # Obliczenie początkowej długości trasy
    prev_val = best_val# Zapamiętanie najlepszego dotąd rozwiązania
    optymalized_tab = [best_val]  # Historia długości tras
    n = len(matrix)
    # Przygotowanie listy na obrazy do animacji
    if gif_name:
        gif_filename = f"zad2_images/gifs/{gif_name}.gif"
        images = []

    for i in range(loop_limit):
        if temp < min_heat:
            # Zatrzymanie, jeśli temperatura spadła poniżej progu
            break
            
        # Zamiana dwóch losowych punktów
        points1_choice, points2_choice = get_random_points(all_cords, max(int(n//100),int(n//100*(temp+1))))
        diff =0
        # Obliczenie zmiany długości trasy po zamianie
        for point1,point2 in zip(points1_choice,points2_choice):
            diff += calc_diff_energy(matrix, point1[0], point1[1], point2[0], point2[1],fun_to_optymalize)
        
        # diff = calc_diff_fun(matrix, i_swap1, j_swap1, i_swap2, j_swap2)
        new_val = diff + prev_val

        # Akceptacja zmiany z prawdopodobieństwem zależnym od temperatury
        if np.random.uniform(0, 1) < propability(diff, temp):
            for point1,point2 in zip(points1_choice,points2_choice):
                i_swap1,j_swap1 = point1
                i_swap2,j_swap2 = point2
                matrix[i_swap1][j_swap1], matrix[i_swap2][j_swap2] = matrix[i_swap2][j_swap2], matrix[i_swap1][j_swap1]
            if new_val - best_val < -eps:
                # Aktualizacja najlepszego rozwiązania
                best_val = new_val
                
            prev_val = new_val
        if i in checkpoints_for_gif:
            if gif_name:
                # Dodanie klatki do GIF-a
                images = extend_gif(matrix,images)
        optymalized_tab.append(prev_val)

        # Aktualizacja temperatury zgodnie z wybraną funkcją
        temp = fun_temp_change(temp, fun_change_par, i, temp_start)

    # Zapisanie animacji GIF, jeśli podano nazwę pliku
    if gif_name:
        images[0].save(gif_filename, save_all=True, append_images=images[1:], duration=100, loop=0)

    return matrix, optymalized_tab, best_val

## Wizualizacja różnorodnych funkcji optymalizujących

Utworzone gify, obrazy oraz wykresy funkcji kosztu znajdują się w stosownym folderze o nazwie `zad2_images` posegregowane na odpowidnie rodzaje. W plikach jest łącznie 480 gifów, tyle samo obrazów i wykresów.

Uwaga:

Przetestowałem wcześniej wszystkie funkcje spadku temperatury z pierwszego zadania. Po ich analizie doszedłem do wniosku, że różnice między nimi są niewielkie, a ewentualne rozbieżności jedynie zwiększają poziom szumu. W związku z tym zdecydowałem się wykorzystać funkcję wykładniczą jako podstawową funkcję w wszystkich dalszych wizualizacjach.

Dodatkowo, w celu uzyskania bardziej przejrzystego porównania warunków żarzenia, zdecydowałem się ograniczyć manipulacje parametrami jedynie do temperatury początkowej. Parametr ten dobrze obrazuje wpływ większej zmienności na ogólny przebieg procesu.

W celu zautomatyzowania procesu tworzenia obrazów, zaimplementowałem funkcję `implement_fun_for_matrix`, która generuje obrazy na podstawie różnych parametrów: prawdopodobieństwa pojawienia się czarnego punktu (`t_delta`), dopuszczalnej liczby iteracji (`t_iter`), rozmiaru obrazu (`t_size`) oraz początkowej temperatury (`t_start_temp`). Funkcja ta pozwala na eksperymentowanie z różnymi ustawieniami, co umożliwia uzyskanie obrazów o różnych cechach w zależności od zadanych parametrów.

In [ ]:
def implement_fun_for_matrix(fun_to_optymalize, name):
    t_delta = [0.1,0.3,0.4,0.6]
    t_iter = [100000, 500000]
    t_size = [128,256,512,1024]
    t_start_temp = [1, 30]
    for iter in t_iter:
            for delta in t_delta:
                for size in t_size:
                    for start_temp in t_start_temp:
                        if start_temp == 30 and iter == 500000: continue
                        matrix, t, _ = create_cristal(np.random.choice([0, 1], size=(size, size), p=[delta, 1-delta]) ,
                                        fun_to_optymalize,
                                        start_temp,
                                        iter,
                                        min_heat=1e-18,
                                        fun_change_par=0.99999,
                                        
                                        gif_name=f"{name}/{name}_{size}x{size}_{delta}_{iter}t{start_temp}")
                        draw_linear_plot(t,f"{name}_{size}x{size}_{delta}_{iter}t{start_temp}",name)
                        draw_plot(matrix,f"{name}_{size}x{size}_{delta}_{iter}t{start_temp}",name)
       

Oznaczenie nazw plików:
- Pierwszy człon określa nazwę funkcji do optymalizacji
- Drugi określa wielkość obrazu
- Trzeci definiuje prawdopodobieństwo pojawienia się czarnego punktu podczas generowania obrazu
- Czwarty pokazuje liczbe przeprowadzonych iteracji
- Litera rozdzielająca "t"
- Ostatni człon to temperatura początkowa

## Implementacje funkcji kosztu

## Funckje bazujące na sąsiedztwie danego punktu

### 1. 4-sąsiedztwo

Ta funkcja kara za niegrupowanie się punktów na mapie. Swoim zasięgiem obejmuje tylko tereny bezpośrednio przyległe do danego punktu

In [ ]:
def four_neighborhood_cell_val(M, x, y, val):
    n = len(M)
    total = 0
    for d in [-1, 1]:
        nx, ny = x + d, y + d
        if 0 <= nx < n and 0 <= ny < n:
            if M[nx][y] != val:
                total += 1
            if M[x][ny] != val:
                total += 1 
    return total

Tworzenie gifów i obrazów

In [ ]:
implement_fun_for_matrix(four_neighborhood_cell_val, "4_neighborhood")

Wizualizacje

$δ = $ 0.4, iter = 500000, temp_start = 1

![alt text](zad2_images/images/4_neighborhood/4_neighborhood_128x128_0.4_500000t1.png.png)

size = 256, $δ = $ 0.6, iter = 500000, temp_start = 1

![alt text](zad2_images/images/4_neighborhood/4_neighborhood_256x256_0.6_500000t1.png.png)

size = 1024, $δ = $ 0.3, iter = 500000, temp_start = 1

![alt text](zad2_images/images/4_neighborhood/4_neighborhood_1024x1024_0.3_500000t1.png.png)

Czarne punkty wykazują wyraźną tendencję do formowania skupisk, które przybierają postać mniejszych lub większych struktur

### 2. 8-sąsiedztwo

Ta funkcja penalizuje brak grupowania punktów na mapie. Jest zbliżona do koncepcji 4-sąsiedztwa, jednak w odróżnieniu od niego, jej zasięg obejmuje obszary oddalone o jedną jednostkę w metryce maksimum.

In [ ]:
def eight_neighborhood_cell_val(M, x, y, val):
    n = len(M)
    total = 0
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue  # pomijamy środek
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n:
                if M[nx][ny] != val:
                    total += 1
    return total


Tworzenie gifów i obrazów

In [ ]:
implement_fun_for_matrix(eight_neighborhood_cell_val, "8_neighborhood")

Wizualizacje i gify

size = 128, $δ = $ 0.6, iter = 500000, temp_start = 1

![alt text](zad2_images/images/8_neighborhood/8_neighborhood_128x128_0.6_500000t1.png.png)

size = 512, $δ = $ 0.6, iter = 500000, temp_start = 1

![alt text](zad2_images/images/8_neighborhood/8_neighborhood_512x512_0.6_500000t1.png.png)

size = 1024, $δ = $ 0.6, iter = 500000, temp_start = 1

![alt text](zad2_images/images/8_neighborhood/8_neighborhood_1024x1024_0.6_500000t1.png.png)

Zaobserwowane skupiska punktów są większe w porównaniu do 4-sąsiedztwa, co wynika z faktu, że premiowane jest nie tylko bezpośrednie przyleganie, ale także szersze sąsiedztwo, obejmujące punkty oddalone o jedną jednostkę w metryce maksimum.

## 3. 16-sąsiedztwo

Ta funkcja penalizuje brak grupowania punktów na mapie. Jest podobna do 4-sąsiedztwa, ale jej zasięg obejmuje obszary oddalone o dwie jednostki w metryce manhatańskiej, co pozwala uwzględnić szersze sąsiedztwo punktów.

In [282]:
def sixteen_neighborhood_cell_val(M, x, y, val):
    n = len(M)
    total = 0
    # Rozważamy 16 sąsiednich komórek wokół (x, y)
    for dx in [-2, -1, 0, 1, 2]:  # Umożliwiamy przesunięcia o 2 jednostki
        for dy in [-2, -1, 0, 1, 2]:
            if abs(dx) == 2 and abs(dy) == 2:continue
            if dx == 0 and dy == 0:
                continue  # Pomijamy środek
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n:  # Sprawdzamy, czy sąsiedzi mieszczą się w obrębie macierzy
                if M[nx][ny] != val:  # Sprawdzamy, czy różni się od wartości 'val'
                    total += 1
    return total

Tworzenie gifów i obrazów

In [ ]:
implement_fun_for_matrix(sixteen_neighborhood_cell_val, "16_neighborhood")

size = 128, $δ = $ 0.6, iter = 100000, temp_start = 1

![alt text](zad2_images/images/16_neighborhood/16_neighborhood_128x128_0.6_100000t1.png.png)

size = 256, $δ = $ 0.4, iter = 500000, temp_start = 1

![alt text](zad2_images/images/16_neighborhood/16_neighborhood_256x256_0.4_500000t1.png.png)

size = 1024, $δ = $ 0.3, iter = 500000, temp_start = 1

![alt text](zad2_images/images/16_neighborhood/16_neighborhood_1024x1024_0.3_500000t1.png.png)

Dla grup tworzonych przez tę funkcję obserwujemy wyraźną nieregularność na ich brzegach. Wiele punktów nie przylega bezpośrednio do innych, co prowadzi do interesującego efektu "poszarpania" na krawędziach grup. Jest to wynikiem zastosowanego kryterium, w którym punkt uznawany za „dobry” może znajdować się również w odległości dwóch pól, co wpływa na kształt i strukturę tych grup.

## 4. Vertical

Ta funkcja nakłada karę za obecność sąsiedztwa w kierunku pionowym.

In [284]:
# kolory lubią być koło swoich
def vertical_cell_val(M, x, y, val):
    n = len(M)
    total = 0
    for d in [-1, 1]:
        nx = x + d
        if 0 <= nx < n:
            if M[nx][y] != val:
                total += 1
    return total


Wizualizacje i gify

In [ ]:
implement_fun_for_matrix(vertical_cell_val, "vertical")

size = 128, $δ = $ 0.3, iter = 500000, temp_start = 1

![alt text](zad2_images/images/vertical/vertical_128x128_0.3_500000t1.png.png)

size = 256, $δ = $ 0.4, iter = 500000, temp_start = 1

![alt text](zad2_images/images/vertical/vertical_256x256_0.4_500000t1.png.png)

size = 512, $δ = $ 0.1, iter = 500000, temp_start = 1

![alt text](zad2_images/images/vertical/vertical_512x512_0.1_500000t1.png.png)

Kryształy formują strukturę przypominającą drewno, jednak w przypadku większych obrazów ten efekt staje się coraz mniej wyraźny. Zamiast tego, dostrzegamy jedynie pojedyncze obszary, które zostały zorganizowane w sposób zwertykalizowany.

## 5. Szachownica

Funkcja ta nakłada karę za obecność sąsiadów o różnych kolorach po przekątnej. Jej celem jest wymuszenie struktury, która przypomina klasyczną szachownicę, gdzie sąsiadujące pola mają naprzemiennie różne kolory, zarówno w poziomie, jak i w pionie, z wyraźnie zaznaczoną regularnością w układzie.

In [292]:
# kolory lubią być koło swoich
def small_chess_cell_val(M, x, y, val):
    n = len(M)
    total = 0
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 or dy == 0:
                continue  # pomijamy środek
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n:
                if M[nx][ny] != val:
                    total += 1
    return total


Wizualizacja i gify

In [ ]:
implement_fun_for_matrix(small_chess_cell_val, "chess")

size = 128, $δ = $ 0.6, iter = 500000, temp_start = 1

![alt text](zad2_images/images/chess/chess_128x128_0.6_500000t1.png.png)

size = 128, $δ = $ 0.3, iter = 500000, temp_start = 1

![alt text](zad2_images/images/chess/chess_128x128_0.3_500000t1.png.png)

size = 256, $δ = $ 0.6, iter = 500000, temp_start = 1

![alt text](zad2_images/images/chess/chess_256x256_0.6_500000t1.png.png)

size = 512, $δ = $ 0.4, iter = 500000, temp_start = 1

![alt text](zad2_images/images/chess/chess_512x512_0.4_500000t1.png.png)

Co interesujące, wygenerowane obrazy nie tyle przywodzą na myśl klasyczną szachownicę, ile raczej przypominają mapę topograficzną. Wydaje się, że obraz składa się z trzech dominujących kolorów: czarnego, białego i szarego. W rzeczywistości, szary kolor powstaje w wyniku interakcji białego i czarnego, tworząc wzór przypominający szachownicę. Dodając do tego tendencję do grupowania obiektów, wynikającą z funkcji kosztu, otrzymujemy estetyczny, „trójkolorowy” obraz o wyraźnie zdefiniowanych strukturach.

## 6. Labirynt

Funkcja ta wprowadza restrykcje, które wymagają, aby punkt najbliżej przekątnej miał inny kolor niż punkt znajdujący się o jedno pole dalej, który powinien być tego samego koloru. Tego rodzaju zasady prowadzą do powstania struktury przypominającej labirynt, w której kolory tworzą złożoną sieć, przypominającą ścieżki i ściany w labiryncie.

In [288]:
# kolory lubią być koło swoich
def lab_cell_val(M, x, y, val):
    n = len(M)
    total = 0
    for dx in [-2,-1, 0, 1,2]:
        for dy in [-2,-1, 0, 1,2]:
            if (dx == 0 and dy == 0) or abs(dx) != abs(dy):
                continue  # pomijamy środek
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n:
                if M[nx][ny] != val:
                    if abs(dx) == 1:
                        total -= 1
                    else:
                        total += 1
    return total

Wizualizacje i gify

In [289]:
implement_fun_for_matrix(lab_cell_val, "lab")

size = 128, $δ = $ 0.6, iter = 500000, temp_start = 1

![alt text](zad2_images/images/lab/lab_128x128_0.6_500000t1.png.png)

size = 256, $δ = $ 0.4, iter = 500000, temp_start = 1

![alt text](zad2_images/images/lab/lab_256x256_0.4_500000t1.png.png)

size = 1024, $δ = $ 0.4, iter = 500000, temp_start = 1

![alt text](zad2_images/images/lab/lab_1024x1024_0.4_500000t1.png.png)

Jak widzimy, nawet dla dużych obrazów widać wyraźnie ta strukturę. Jest ona wizualnie ciekawa

## Funkcje bazujące na ułożeniu punktu na mapie

### 7. Center

Wprowadzam prostą zasadę, zgodnie z którą punkty czarne mają tendencję do unikania centralnych obszarów. Funkcja kosztu jest proporcjonalna do odległości od środka, penalizując umiejscowienie czarnych punktów zbliżonych do centrum, co w efekcie prowadzi do ich rozmieszczania się na obrzeżach.

In [290]:
# kolory lubią być koło swoich
def center_ceil_val(M, x, y, val):
    n = len(M)
    sr = n//2
    return math.sqrt((x-sr)**2 + (y-sr)**2) if val == 1 else 0


Wizualizacje i gify

In [295]:
implement_fun_for_matrix(center_ceil_val, "center")

size = 128, $δ = $ 0.3, iter = 500000, temp_start = 1

![alt text](zad2_images/images/center/center_128x128_0.3_500000t1.png.png)

size = 256, $δ = $ 0.3, iter = 500000, temp_start = 1

![alt text](zad2_images/images/center/center_256x256_0.3_500000t1.png.png)

size = 1024, $δ = $ 0.1, iter = 500000, temp_start = 1

![alt text](zad2_images/images/center/center_1024x1024_0.1_500000t1.png.png)

Jak widać na powyższych ilustracjach, na wierzchołkach i krawędziach obrazu tworzy się wyraźne zagęszczenie czarnych punktów. To zjawisko jest wynikiem wprowadzonej zasady, która penalizuje obecność czarnych punktów w centralnych częściach obrazu, kierując je ku brzegom.

### 8. 8 pasów

Funkcja ma na celu podział punktów na osiem poziomych obszarów na mapie, przypisując je do odpowiednich kategorii w zależności od ich pozycji wzdłuż osi poziomej. 

In [ ]:
def eight_streaks_cell_value(M, x, y, val):
    n = len(M)
    kw = n//8
    if (x<kw or 2*kw<x<3*kw or 4*kw<x<5*kw or 6*kw<x<7*kw):
        if val == 1:
            return 1
    else:
        if val == 0:
            return 1
    return 0

Wizualizacje i gify

In [ ]:
implement_fun_for_matrix(eight_streaks_cell_value, "streaks")

size = 128, $δ = $ 0.4, iter = 500000, temp_start = 1

![alt text](zad2_images/images/streaks/streaks_128x128_0.4_500000t1.png.png)


size = 256, $δ = $ 0.4, iter = 500000, temp_start = 1

![alt text](zad2_images/images/streaks/streaks_256x256_0.4_500000t1.png.png)

size = 1024, $δ = $ 0.4, iter = 500000, temp_start = 1

![alt text](zad2_images/images/streaks/streaks_1024x1024_0.4_500000t1.png.png)

Jak widać powyżej, na obrazach wyraźnie widoczne są pasy, które wynikają z zastosowanego podziału na poziome obszary.

### 9. Big chess

Funkcja ma na celu podzielenie obszaru na mniejsze podobszary, które będą tworzyć strukturę przypominającą dużą szachownicę.

In [ ]:
# 8 pasów
def get_value_like_matrix_bigchess(M, x, y, val):
    n = len(M)
    kw = n//8
    if (x<kw or 2*kw<x<3*kw or 4*kw<x<5*kw or 6*kw<x<7*kw)/
    and (y<kw or 2*kw<y<3*kw or 4*kw<y<5*kw or 6*kw<y<7*kw):
        if val == 1:
            return 1
    else:
        if val == 0:
            return 1
    return 0

Wizualizacje i gify

In [ ]:
implement_fun_for_matrix(get_value_like_matrix_bigchess, "big_chess")

size = 128, $δ = $ 0.3, iter = 500000, temp_start = 1

![alt text](zad2_images/images/big_chess/big_chess_128x128_0.3_500000t1.png.png)

size = 256, $δ = $ 0.1, iter = 500000, temp_start = 1

![alt text](zad2_images/images/big_chess/big_chess_256x256_0.1_500000t1.png.png)

size = 1024, $δ = $ 0.3, iter = 500000, temp_start = 1

![alt text](zad2_images/images/big_chess/big_chess_1024x1024_0.3_500000t1.png.png)

Układ szachownicy jest widoczny na każdym rozmiarze obrazu.

## Dodatkowy czerwony kolor

### Modyfikacja poprzednich funkcji wizualizujących wyniki

W celu wprowadzenia dodatkowego koloru potrzebna jest lekka modyfikacja poniższych funkcji

In [ ]:

def create_image_from_array(matrix):
    h, w = matrix.shape
    img = np.zeros((h, w, 3), dtype=np.uint8)  # 3 kanały: RGB
    img[matrix == 0] = [255, 255, 255]  # białe tło
    img[matrix == 1] = [0, 0, 0]        # czarne punkty
    img[matrix == 2] = [255, 0, 0]      # czerwony
    return Image.fromarray(img, mode='RGB')

In [ ]:
def draw_plot(matrix, path=None, folder_name=''):
    n = len(matrix)
    img = Image.new('RGB', (n, n))  # RGB - kolorowy obraz

    pixels = img.load()
    for i in range(n):
        for j in range(n):
            val = matrix[i, j]
            if val == 0:
                color = (255, 255, 255)  # biały
            elif val == 1:
                color = (0, 0, 0)        # czarny
            elif val == 2:
                color = (255, 0, 0)      # czerwony
            else:
                color = (128, 128, 128)  # kolor awaryjny dla nieznanych wartości
            pixels[j, i] = color

    # Upewnij się, że folder istnieje
    output_path = f"zad2_images/images/{folder_name}"
    # Zapisz obraz
    img.save(f"{output_path}/{path}.png")

In [ ]:
def extend_gif(matrix,images):
    n = len(matrix)
    img = Image.new('RGB', (n, n))  # RGB - kolorowy obraz

    pixels = img.load()
    for i in range(n):
        for j in range(n):
            val = matrix[i, j]
            if val == 1:
                color = (255, 255, 255)  # biały
            elif val == 0:
                color = (0, 0, 0)        # czarny
            elif val == 2:
                color = (255, 0, 0)      # czerwony
            else:
                color = (128, 128, 128)  # kolor awaryjny dla nieznanych wartości
            pixels[j, i] = color
            
    images.append(img) # 0 to biały, 1 to czarny
    return images

### Funkcja kosztu

Wprowadzam funkcję kosztu, którą można streścić jako „czarne lubią czerwone”. Funkcja ta bierze pod uwagę sąsiedztwo w obrębie trzech jednostek w metryce maksimum, premiując układy, w których czarne punkty znajdują się w pobliżu czerwonych. 

In [ ]:
def get_black_like_red(M, x, y, val):
    # kolory lubią być koło swoich
    n = len(M)
    total = 0
    for dx in [-3, 0, 3]:
        for dy in [-3, 0, 3]:
            if dx == 0 and dy == 0:
                continue  # pomijamy środek
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n:
                if M[nx][ny] == 2 and val ==1:
                    total += 1
                if M[nx][ny] == 1 and val ==2:
                    total += 1
    return total


Wizualizacje i gify

In [ ]:
def implement_fun_for_matrix_red(fun_to_optymalize, name):
    t_delta = [0.1,0.3,0.4]
    t_iter = [500000]
    t_size = [128,256,512,1024]
   
    for iter in t_iter:
            for delta in t_delta:
                for size in t_size:
                    old_array = np.random.choice([0, 1, 2], size=(size//2, size//2), p=[delta, 1-delta-0.03,0.03])
                    new_array = np.random.choice([0, 1], size=(size, size), p=[delta, 1 - delta])
                    new_array[size//4:3*size//4, size//4:3*size//4] = old_array
                    matrix, t, _ = create_cristal(new_array ,
                                    fun_to_optymalize,
                                    1,
                                    iter,
                                    min_heat=1e-18,
                                    fun_change_par=0.99999,
                                    
                                    gif_name=f"{name}/{name}_{size}x{size}_{delta}_{iter}")
                    draw_linear_plot(t,f"{name}_{size}x{size}_{delta}_{iter}",name)
                    draw_plot(matrix,f"{name}_{size}x{size}_{delta}_{iter}.png",name)
       
implement_fun_for_matrix_red(get_black_like_red, "red_like")

size = 128, $δ = $ 0.1, iter = 500000, temp_start = 1

![alt text](zad2_images/images/red_like/red_like_128x128_0.1_500000.png.png)

size = 256, $δ = $ 0.1, iter = 500000, temp_start = 1

![alt text](zad2_images/images/red_like/red_like_256x256_0.1_500000.png.png)

size = 512, $δ = $ 0.1, iter = 500000, temp_start = 1

![alt text](zad2_images/images/red_like/red_like_512x512_0.1_500000.png.png)

Widzimy, że tworzą się skupiska wokół statycznych czerwonych punktów

# Analiza wyników dla róznych parametrów

Przetestowałem generowanie funkcji dla różnej wartości początkowej temperatury. Jest to parametr, który najmocniej wpływa na sposób generowania oraz najłatwiej będzie go omówić

Przykładowe obraz dla temp_start = 1

Chess, size = 128, $δ = $ 0.4, iter = 100000

![alt text](zad2_images/images/chess/chess_128x128_0.4_100000t1.png.png)

big_chess, size = 512, $δ = $ 0.6, iter = 100000

![alt text](zad2_images/images/big_chess/big_chess_512x512_0.6_100000t1.png.png)

#### Obrazy dla temp_start = 30

Chess, size = 128, $δ = $ 0.4, iter = 100000

![alt text](zad2_images/images/chess/chess_128x128_0.4_100000t30.png.png)

big_chess, size = 512, $δ = $ 0.6, iter = 100000

![alt text](zad2_images/images/big_chess/big_chess_512x512_0.6_100000t30.png.png)

Po przeanalizowaniu wygenerowanych materiałów można stwierdzić, że wysoka temperatura wprowadza znaczące zaszumienie do obrazu. Patterny i struktury, które są wyraźnie widoczne przy niższej temperaturze, stają się praktycznie niezauważalne i rozmywają się w chaosie szumu przy wyższych wartościach temperatury.

Ale jest jedna struktura, której jakość nie pogorszyła się wraz z dużym wzrostem temperatury początkowej. Jest nią struktura Center

Przykładowe obraz dla temp_start = 1

Center, size = 256, $δ = $ 0.1, iter = 100000

![alt text](zad2_images/images/center/center_256x256_0.1_100000t1.png.png)

Obrazy dla temp_start = 30

Center, size = 256, $δ = $ 0.1, iter = 100000

![alt text](zad2_images/images/center/center_256x256_0.1_100000t30.png.png)

Oba obrazy są do siebie bardzo podobne, z drobną przewagą obrazu uzyskanego przy temp_start = 1, biorąc pod uwagę liczbę pojedynczych elementów w centrum. W tym przypadku zaszumienie nie wpłynęło negatywnie na obraz, a wręcz dodało mu pewnego kolorytu, wzbogacając całość o interesujące detale.

# Wykresy funkcji optymalizowanych

Przygladnijmy się wykresom funkcji optymalizowanych

Widać dwie wyraźne tendencje:

- Dla wysokiej temperatury, funkcja optymalizowana przyjmuje bardzo nieregularny przebieg, co może wskazywać na większą podatność na szum i brak stabilności w procesie optymalizacji.

- Dla niskiej temperatury, wykres staje się bardziej gładki, co sugeruje, że algorytm koncentruje się na bardziej stabilnych i precyzyjnych rozwiązaniach, jednak może również utknąć w lokalnych minimach, przez co optymalizacja staje się mniej dynamiczna.

Dodatkowo, można zauważyć, że im mniejsza mapa, tym wykres staje się bardziej chaotyczny. W przypadku mniejszych przestrzeni optymalizacyjnych, algorytm może napotkać większe trudności w znajdowaniu stabilnych rozwiązań, co prowadzi do bardziej losowych i nieregularnych wyników, zanim osiągnie bardziej jednolity stan w miarę zmniejszania temperatury.

Porównanie wykresów dla różnych temperatur:

![alt text](zad2_images/plots/8_neighborhood/8_neighborhood_128x128_0.1_100000t30.png)

![alt text](zad2_images/plots/8_neighborhood/8_neighborhood_128x128_0.1_100000t1.png)

Porównanie dla wzrastających wymiarów obrazu

![alt text](zad2_images/plots/8_neighborhood/8_neighborhood_128x128_0.1_100000t30.png)

![alt text](zad2_images/plots/8_neighborhood/8_neighborhood_256x256_0.1_100000t30.png)

![alt text](zad2_images/plots/8_neighborhood/8_neighborhood_512x512_0.1_100000t30.png)

![alt text](zad2_images/plots/8_neighborhood/8_neighborhood_1024x1024_0.1_100000t30.png)

Wraz ze wzrostem wielkości obrazka, zmienność zanika

Jedynym odstępstwem od tej zasady jest ponownie wykres funkcji dla _center_, który we wszystkich warunkach przyjmuje ten sam charakterystyczny kształt:

![alt text](zad2_images/plots/center/center_512x512_0.3_500000t1.png)

# Podsumowanie

- Algorytm wyżarzania okazał się niezwykle skutecznym narzędziem do generowania struktur przypominających kryształy.

- Najlepsze rezultaty osiągnięto przy zastosowaniu wykładniczej funkcji temperatury.

- Dzięki wykorzystaniu różnych funkcji optymalizacyjnych udało się uzyskać interesujące i zróżnicowane obrazy.

- Niestabilność wynikająca z wysokiej temperatury początkowej została zniwelowana przez rosnące wymiary obrazów.

- Wysoka temperatura początkowa w wielu przypadkach prowadziła do powstania dużego chaosu w obrazie, który ostatecznie zamieniał się w zwykły szum.

- Dla obrazów typu center, zwiększona temperatura początkowa wprowadzała jedynie niewielki poziom szumu, w przeciwieństwie do pozostałych wzorców.

